In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import utils
from models import PyModel
from allmodels import load_mnist_data
from torch.autograd import Variable
import torch

import cifar10_input
from cifar_model import Model
import foolbox
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

/home/yusu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class blackbox:
    def __init__(self,model):
        self.model = model
        
    def attack_untargeted(self, x0, y0, alpha = 2,num_query = 100, beta = 0.005, iterations = 1000):
        """ Attack the original image and return adversarial example
            model: (pytorch model)
            alpha: learning rate 
            beta: learning rate
            train_dataset: set of training data
            (x0, y0): original image
        """
        
        if (self.model.predict_label(x0)!= y0):
            print("Fail to classify the image. No need to attack.")
            return x0,0
    
        num_directions = 200
        num_query = 10
        best_theta, g_theta = None, float('inf')
        query_count = 0
        comp_theta = 0
        current_best = float('inf')
        
        timestart = time.time()
        for i in range(num_directions):
            theta = torch.randn(x0.shape).type(torch.FloatTensor)
            initial_lbd = torch.norm(theta)
            theta = theta/torch.norm(theta)
            if self.model.predict_label(x0+np.array(initial_lbd*theta)) != y0:
                lbd,comp_dec = self.fine_grained_binary_search_fix(x0,y0,theta,initial_lbd,g_theta,current_best,num_query)
                if comp_dec > comp_theta:
                    comp_theta = comp_dec
                    best_theta,g_theta = theta,lbd
                    print("--------> Found abs-distortion %.4f with 10 queries" % g_theta)
                    print("--------> Found comp-distortion %.4f with 10 queries" % comp_dec)
            timeend = time.time()
        print("==========> Found best distortion %.4f in %.4f seconds" % (g_theta, timeend-timestart))
        query_count = (num_directions+1)*num_query
        print("type of best_theta", type(best_theta))
        print("type of best_theta", type(g_theta))
        lbd,count = self.fine_grained_binary_search( x0, y0, best_theta, g_theta, current_best)
        g_theta = lbd
        query_count += count
                    
        
        #timestart = time.time()
        #for i in range(num_directions):
        #    theta = torch.randn(x0.shape).type(torch.FloatTensor)
        #    #print(theta.size())
        #    initial_lbd = torch.norm(theta)
        #    theta = theta/torch.norm(theta)
        #    if self.model.predict_label(x0+np.array(initial_lbd*theta)) != y0:
        #        lbd, count = self.fine_grained_binary_search( x0, y0, theta, initial_lbd, g_theta)
        #        query_count += count
        #        if lbd < g_theta:
        #            best_theta, g_theta = theta,lbd
        #            print("--------> Found distortion %.4f" % g_theta)
        #    timeend = time.time()
        #print("==========> Found best distortion %.4f in %.4f seconds using %d queries" % (g_theta, timeend-timestart, query_count))
        
            
        g1 = 1.0
        theta, g2 = best_theta.clone(), g_theta
        torch.manual_seed(0)
        opt_count = 0
        stopping = 0.01
        prev_obj = 100000
        
        for i in range(iterations):
            if g_theta < 1:
                print("====================query number after distortion < 1 =======================: ",opt_count)
                break
            #print("n_query:",opt_count)
            #print("best distortion:", g_theta)
            print("iteration:", i )
            gradient = torch.zeros(theta.size())
            q = 10
            min_g1 = float('inf')
            for _ in range(q):
                u = torch.randn(theta.size()).type(torch.FloatTensor)
                u = u/torch.norm(u)
                ttt = theta+beta * u
                ttt = ttt/torch.norm(ttt)
                #print("before 1st binary")
                g1, count = self.fine_grained_binary_search_local( x0, y0, ttt, initial_lbd = g2, tol=beta/500)
                #print("after 1st binary")
                opt_count += count
                gradient +=  (g1-g2)/beta * u
                #print("norm of gradient:", np.linalg.norm(gradient))
                if g1 < min_g1:
                    min_g1 = g1
                    min_ttt = ttt
            gradient = 1.0/q * gradient
    
            if (i+1)%50 == 0:
                
                print("Iteration %3d: g(theta + beta*u) = %.4f g(theta) = %.4f distortion %.4f num_queries %d" % (i+1, g1, g2, torch.norm(g2*theta), opt_count))
                if g2 > prev_obj-stopping:
                    break
                prev_obj = g2
    
            min_theta = theta
            min_g2 = g2
        
            for _ in range(15):
                #print("enter first for loop")
                new_theta = theta - alpha * gradient
                new_theta = new_theta/torch.norm(new_theta)
                #print("before 2nd binary")
                new_g2, count = self.fine_grained_binary_search_local( x0, y0, new_theta, initial_lbd = min_g2, tol=beta/500)
                #print("after 2nd binary")
                opt_count += count
                alpha = alpha * 2
                #print("alpha in the first for loop is: ",alpha)
                if new_g2 < min_g2:
                    min_theta = new_theta 
                    min_g2 = new_g2
                else:
                    break
    
            if min_g2 >= g2:
                for _ in range(15):
                    #print("enter second for loop")
                    alpha = alpha * 0.95
                    new_theta = theta - alpha * gradient
                    new_theta = new_theta/torch.norm(new_theta)
                    #print("before 3rd binary")
                    new_g2, count = self.fine_grained_binary_search_local( x0, y0, new_theta, initial_lbd = min_g2, tol=beta/500)
                    #print("after 3rd binary")
                    opt_count += count
                    #print("alpha in the second for loop is: ",alpha)
                    if new_g2 < g2:
                        min_theta = new_theta 
                        min_g2 = new_g2
                        break
    
            if min_g2 <= min_g1:
                theta, g2 = min_theta, min_g2
            else:
                theta, g2 = min_ttt, min_g1
    
            if g2 < g_theta:
                best_theta, g_theta = theta.clone(), g2
            
            #print(alpha)
#            print("%3d th iteration" % i)
            #print("current alpha:",alpha)
            if alpha < 1e-4:
                alpha = 1.0
                #print("Warning: not moving, g2 %lf gtheta %lf" % (g2, g_theta))
                beta = beta * 0.1
                if (beta < 0.0005):
                    break
    
        #target = model.predict(x0 + g_theta*best_theta)
        
        #timeend = time.time()
        #print("\nAdversarial Example Found Successfully: distortion %.4f target %d queries %d \nTime: %.4f seconds" % (g_theta, target, query_count + opt_count, timeend-timestart))
        print("baseline")
        print("best distortion :", g_theta)
        print("number of queries :", opt_count+query_count)
        return np.array(g_theta*best_theta), opt_count+query_count
   
    def fine_grained_binary_search_fix(self,x0,y0,theta, initial_lbd = 1.0, tol=1e-5,current_best = float('inf'),num_query = 100):
        nquery = 0
        if initial_lbd > current_best: 
            if self.model.predict_label(x0+ np.array(current_best*theta)) == y0:
                nquery += 1
                return float('inf'), nquery
            lbd = current_best
        else:
            lbd = initial_lbd
    
        lbd_hi = lbd
        lbd_lo = 0.0
    
        while (lbd_hi - lbd_lo) > 1e-5:
            lbd_mid = (lbd_lo + lbd_hi)/2.0
            nquery += 1
            if nquery > num_query:
                break
            if self.model.predict_label(x0 + np.array(lbd_mid*theta)) != y0:
                lbd_hi = lbd_mid
            else:
                lbd_lo = lbd_mid
        comp_dec = (initial_lbd - lbd_hi)/initial_lbd
       # print("number of query before return for this direction:",nquery)
        return lbd_hi,comp_dec
    
    def fine_grained_binary_search_local(self, x0, y0, theta, initial_lbd = 1.0, tol=1e-5):
        nquery = 0
        lbd = initial_lbd
         
        if self.model.predict_label(x0+np.array(lbd*theta)) == y0:
            lbd_lo = lbd
            lbd_hi = lbd*1.01
            nquery += 1
            while self.model.predict_label(x0+np.array(lbd_hi*theta)) == y0:
                lbd_hi = lbd_hi*1.01
                nquery += 1
                if lbd_hi > 20:
                    return float('inf'), nquery
        else:
            lbd_hi = lbd
            lbd_lo = lbd*0.99
            nquery += 1
            while self.model.predict_label(x0+np.array(lbd_lo*theta)) != y0 :
                lbd_lo = lbd_lo*0.99
                nquery += 1
    
        while (lbd_hi - lbd_lo) > tol:
            #print("while loop")
            lbd_mid = (lbd_lo + lbd_hi)/2.0
            nquery += 1
            if self.model.predict_label(x0 + np.array(lbd_mid*theta)) != y0:
                lbd_hi = lbd_mid
            else:
                lbd_lo = lbd_mid
        return lbd_hi, nquery
    
    def fine_grained_binary_search(self, x0, y0, theta, initial_lbd, current_best):
        nquery = 0
        if initial_lbd > current_best: 
            if self.model.predict_label(x0+ np.array(current_best*theta)) == y0:
                nquery += 1
                return float('inf'), nquery
            lbd = current_best
        else:
            lbd = initial_lbd
        
        ## original version
        #lbd = initial_lbd
        #while model.predict(x0 + lbd*theta) == y0:
        #    lbd *= 2
        #    nquery += 1
        #    if lbd > 100:
        #        return float('inf'), nquery
        
        #num_intervals = 100
    
        # lambdas = np.linspace(0.0, lbd, num_intervals)[1:]
        # lbd_hi = lbd
        # lbd_hi_index = 0
        # for i, lbd in enumerate(lambdas):
        #     nquery += 1
        #     if model.predict(x0 + lbd*theta) != y0:
        #         lbd_hi = lbd
        #         lbd_hi_index = i
        #         break
    
        # lbd_lo = lambdas[lbd_hi_index - 1]
        lbd_hi = lbd
        lbd_lo = 0.0
    
        while (lbd_hi - lbd_lo) > 1e-5:
            lbd_mid = (lbd_lo + lbd_hi)/2.0
            nquery += 1
            if self.model.predict_label(x0 + np.array(lbd_mid*theta)) != y0:
                lbd_hi = lbd_mid
            else:
                lbd_lo = lbd_mid
        return lbd_hi, nquery

In [3]:
sess = tf.InteractiveSession()

In [4]:
cifar = cifar10_input.CIFAR10Data("../cifar10_data")
orig_model = Model("../models/standard/", tiny=False, mode='eval', sess=sess)
model = PyModel(orig_model,sess,[0.0,255.0])
images = cifar.eval_data.xs[20:1000]/255
labels = cifar.eval_data.ys[20:1000]
pre_labs = []
count = 0
for i in range(20):
    pre_lab = model.predict_label(images[i])
    pre_labs.append(pre_lab)
    if labels[i] == pre_lab: 
        count+=1
print("original labels:", labels[:20])
print("predicted labels:",pre_labs)
print("accuracy of 20 images :", count/20)

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
INFO:tensorflow:Restoring parameters from ../models/standard/checkpoint-68000
original labels: [7 0 4 9 5 2 4 0 9 6 6 5 4 5 9 2 4 1 9 5]
predicted labels: [array([7]), array([0]), array([4]), array([9]), array([5]), array([2]), array([4]), array([0]), array([9]), array([6]), array([6]), array([5]), array([4]), array([5]), array([9]), array([2]), array([4]), array([1]), array([9]), array([5])]
accuracy of 20 images : 1.0


In [ ]:

image = cifar.eval_data.xs[:2]/255# np.array


label = cifar.eval_data.ys[:2]

attack = blackbox(model)

#print("original label is:",label)
#timestart = time.time()
#print("predicted label on clean data is: ", model.predict_label(test_img[0]))
#timeend = time.time()
#print("time consuming: ", timeend-timestart)

dist = []
count = []
for i in range(3):
    print("================attacking image ",i+1,"=======================")
    mod,queries = attack.attack_untargeted(image[i+1],label[i+1],alpha = 4, beta = 0.005, iterations = 1000)
    dist.append(np.linalg.norm(mod))
    count.append(queries)


================attacking image  1 =======================
--------> Found abs-distortion 5.5957 with 10 queries
--------> Found comp-distortion 0.9004 with 10 queries
--------> Found abs-distortion 4.2933 with 10 queries
--------> Found comp-distortion 0.9229 with 10 queries
--------> Found abs-distortion 4.1497 with 10 queries
--------> Found comp-distortion 0.9248 with 10 queries
--------> Found abs-distortion 3.1693 with 10 queries
--------> Found comp-distortion 0.9424 with 10 queries
==========> Found best distortion 3.1693 in 25.2583 seconds
type of best_theta <class 'torch.Tensor'>
type of best_theta <class 'torch.Tensor'>
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8


In [6]:
index = np.nonzero(count)
index= list(index)[0].tolist()

#index2 = np.nonzero(count)
#index2 = list(index2)[0].tolist()


avg_distortion = np.mean(np.array(dist)[index])
avg_count = np.mean(np.array(count)[index])
print("the average distortion for %2d images :"%(len(index)),avg_distortion)
for i in dist:
    print(i)
print("the number of queries for %2d images :"%(len(index)), avg_count)
for j in count:
    print(j)
#print("the average distortion of 10 images is:", avg_distortion)
#print("the average queries of 10 images:", avg_distortion)

the average distortion for  3 images : 0.9190309
0.99744064
0.9868541
0.772798
the number of queries for  3 images : 123178.0
348055
11452
10027
